In [ ]:
!pip install --upgrade xlrd

In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.integrate import quad

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel(r'/content/drive/MyDrive/Data_BlackKoffer_intern/DATA1/37.xls',sheet_name='results',header=None)
df

In [ ]:
def f1(sigma,t):   # t is the timestamp
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):  # we want it to go from start ... t, that's why used t+1                  
    i = k-1  # id of data point, since k is id+1 (k is in column no.0 which is id+1)
    val = math.exp(-df[1][i]*df[1][i] / (2*sigma*sigma))
    val = val / (math.sqrt(2*math.pi)*sigma)
    result = result * val
  return result

In [ ]:
def f2(t):
  result = 1
  for k in range(20*math.floor((t-1)/20)+1, t+1):                                   # 
    i = k-1
    val = math.pi*(1 + df[1][i]*df[1][i])
    val = 1 / val
    result = result * val
  return result

In [ ]:
def m1(t):
  val = lambda t : quad(f1, 0.1,2, args=(t,))
  ans,err = val(t)
  return (1/1.9)*ans

In [ ]:
def m2(t):
  return f2(t)

In [ ]:
Pm1 = 0.5
Pm2 = 0.5

In [ ]:
def Pm1Xt(t):
  val = Pm1 * m1(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

In [ ]:
def Pm2Xt(t):
  val = Pm2 * m2(t)
  val = val / (Pm1*m1(t) + Pm2*m2(t))
  return val

We win 2dollars on winning bet and loss $40 on lossing bet

In [ ]:
def sigmaapprox(t):
  start = 20*math.floor((t-1)/20)+1                                                 
  end = t
  if start==t: 
    return 0  # no variance if there is only one data point
  result = 1 / (end-start+1)
  val = 0
  for k in range(start,end+1):
    i = k-1
    val += df[1][i]*df[1][i]
  result = result * val
  result = math.sqrt(result)
  
  return result

In [ ]:
def pg(t):
  sigma = sigmaapprox(t)
  if sigma==0:
    return 0
  def integrand(x):
      result = 1 / (sigma*math.sqrt(2*math.pi))
      result = result * math.exp(-x*x / (2*sigma*sigma))
      return result
  phi,err = quad(integrand,-float('inf'),4)
  result = 1 - 2*(1-phi)
  
  return result  

In [ ]:
def pc():
  return 0.844

In [ ]:
def compute_expValue(beta, DA, t):
  if t%20==0 or t%20==1: return 0   # t is the first or the last data point of a trial set
  Pg = pg(t)
  Pc = pc()
  if t%20<5: DA=0
  V = Pm1Xt(t-1)*((2+DA)*Pg - (1-Pg)*40) + Pm2Xt(t-1)*((2+DA)*Pc-(1-Pc)*40)
  return V # expected value

We bet on trial t+1 if and only if Vt > 0 

In [ ]:

def predict(beta,DA,starting,ending):
  total = 0
  plist = []  # list of outcomes (bet or not bet)
  expList = []  # list of expected values for each time stamp
  probList = []
  for i in range(starting,ending): 
    # i is the time stamp. 
    Vt = compute_expValue(beta,DA,i) # Computing expected value for timestamp i
    prob = 1 / (1 + math.exp(-beta*Vt))
    expList.append(Vt)
    probList.append(prob)
    if prob>0.5:
      plist.append(1)
      if df[1][i]<=4 and df[1][i]>=-4: # in the dataframe, index of data at time i+1 is i
        total+=2 
      else: 
        total-=40
    else: plist.append(0)        # adding value for timestamp i+1
  return (plist,expList,probList,total)       

Now we train the model (the parameter DA)

In [ ]:
Y = df[5]

In [ ]:
def errorFunc(prediction, Y):  # cost function
  total = 0
  for i in range(3,160):  # i is timestamp
    #total += -(Y[i+160]*math.log(prediction[i-1]) + (1-Y[i+160])*math.log(1-prediction[i-1]))
    total += -(Y[i]*math.log(prediction[i]) + (1-Y[i])*math.log(1-prediction[i]))
  total /= 160

  return total  

In [ ]:
def derivative_cost_function(beta,DA):  # gradient of cost function (used in gradient descent)
  G_DA = 0
  G_b = 0
  plist,expList,probList,total = predict(beta,DA,0,160)
  for i in range(3,160):
    if (i+1)%20<5: continue
    #G_b += (expList[i-1-160]-Y[i])*(expList[i-1-160])
    #G_DA += (expList[i-1-160]-Y[i])*beta*(pg(i-1)*Pm1Xt(i-2) + pc()*Pm2Xt(i-2))
    G_b += (probList[i]-Y[i])*(expList[i])
    G_DA += (probList[i]-Y[i])*beta*(pg(i)*Pm1Xt(i-1) + pc()*Pm2Xt(i-1))
  G_b = G_b/160
  G_DA = G_DA/160 
  return (G_DA,G_b)

gradient descent

In [ ]:
DA = 0.01
beta = 1
alpha = 0.05
i = 0
while True:
  
  if DA>=5: break
  
  if i==0:
    pre_DA = DA
    pre_beta = beta
    i+=1
  else:
    G_DA,G_b = derivative_cost_function(beta,DA)
    DA = DA - alpha*G_DA
    beta = beta - alpha*G_b
    print("DA, beta: ",DA,beta)

    if abs(DA-pre_DA)<1e-3 or abs(beta-pre_beta)<1e-3: break
    else: 
      pre_DA = DA
      pre_beta = beta
    i+=1
if DA<0: DA = 0.01    

Tesing the resulting model on remaining 7 sessions and calculating accuracy score

In [ ]:
plist,expList,probList,total = predict(beta,DA,160,300)   # prediction

print(len(plist))
print(total)

In [ ]:
print(plist)

In [ ]:
total = 0
Y = df[5]
print(Y[12]) 
for i in range(0,140):  # we will have to change it to (0,160) when the dataset has a length of 320
  if (plist[i]==0 and Y[i+160]==0) or (plist[i]==1 and Y[i+160]==1): total+=1
print("The accuracy score of the base model with CbD is: ",total/140*100,"%")   # similarly here 140 -> 160